In [1]:
import import_hack
import numpy as np
import core.steps as steps
from core import data_store as ds
from core import transformer as tr
from core.common.ds_utils import print_ds_items_info
from os.path import join as pjoin
from core.search.exhaustive_searcher import ExhaustiveSearcher
from core.quantization.pq_quantizer import PQQuantizer, restore_from_clusters, build_pq_params_str
from core.metric.symmetric_distance_computer import SymmetricDistanceComputer
%load_ext autoreload
%autoreload 2

In [21]:
base_dir=r'C:\data\computation\brodatz\global_descriptors'

#### Setup global descriptors datastores
descriptors have been computed in global_descriptor__compute_from_image 

In [22]:
histograms_ds = ds.SQLiteTableDataStore(pjoin(base_dir, 'histograms'))
lbphistograms_ds = ds.SQLiteTableDataStore(pjoin(base_dir, 'lbphistograms'))
glcms_ds = ds.SQLiteTableDataStore(pjoin(base_dir, 'glcms'))

# SDC exhaustive search

In [23]:
K_arr = [2 ** i for i in [4,5,6,7,8]]
m_arr = [m for m in [1, 2, 4, 8, 16]]
pq_params_arr = [{'n_clusters': K, 'n_quantizers': m} for K in K_arr for m in m_arr]

In [32]:
def sdc_search(pq_params, descriptors_name, query_descriptors_ds):
    pq_params_str = build_pq_params_str(pq_params)
    quantizer = PQQuantizer(**pq_params)

    centroids_path = pjoin(base_dir, 'centroids', descriptors_name, descriptors_name) + "_" + pq_params_str + '_centroids'
    centroids_ds = ds.SQLiteTableDataStore(centroids_path)
    centroids = ds.get_as_array(centroids_ds)
    pq_quantizer = restore_from_clusters(centroids)
  
    centroids_pairwise_distances_path=pjoin(base_dir, 'centroids_pairwise_distances', descriptors_name, descriptors_name)+ "_" + pq_params_str + '_centroids_pairwise_distances'
    centroids_pairwise_distances_ds=ds.SQLiteTableDataStore(centroids_pairwise_distances_path)
    centroids_pairwise_distances = ds.get_as_array(centroids_pairwise_distances_ds)
    
    metric = SymmetricDistanceComputer(pq_quantizer, centroids_pairwise_distances)
    
    pqcodes_path = pjoin(base_dir, 'pqcodes', descriptors_name, descriptors_name) + "_" + pq_params_str + '_pqcodes'
    pqcodes_ds = ds.SQLiteTableDataStore(pqcodes_path, ndarray_bytes_only=True)
    ids, pqcodes = ds.get_as_array(pqcodes_ds, return_ids=True)
    
    searcher_ = ExhaustiveSearcher(pqcodes, ids, metric=metric)
    n_nearest = 25

    neighbors_ids_filepath = pjoin(base_dir, 'sdc-neighbors-ids', descriptors_name, descriptors_name) + "_" + pq_params_str + '_sdc-neighbors-ids'
    neighbors_ids_ds = ds.CSVDataStore(neighbors_ids_filepath)
    
    steps.search_step(query_descriptors_ds, searcher_, n_nearest, neighbors_ids_ds)

In [33]:
for pq_params in pq_params_arr:
    sdc_search(pq_params, 'histograms', histograms_ds)

In [34]:
for pq_params in pq_params_arr:
    sdc_search(pq_params, 'lbphistograms', lbphistograms_ds)

In [35]:
for pq_params in pq_params_arr:
    sdc_search(pq_params, 'glcms', glcms_ds)